In [1]:
from copy import deepcopy
import os
import math
import shutil
import re
from collections import defaultdict

import pandas as pd
import rispy

In [2]:
dump_dir = '../data/bntl-db-csv-dump-4March2024'

### Authors

In [3]:
authors = pd.read_csv(f"{dump_dir}/publication_authors.csv", header=0)
authors.sample(10)

,publication_id,author_id,sequence
51236,21251,25357,63192
167643,205018,3829,210791
217501,211591,17741,271080
286234,117947,11983,586198
292172,12169,21930,391829
41073,61364,33442,50567
39043,58481,21135,48113
145966,236140,7081,182984
316120,307877,70254,641440
328,290445,46612,497415


In [4]:
pubid2authorid = defaultdict(list)
for pub_id, author_id in zip(authors['publication_id'], authors['author_id']): # komen auteurs er in de juiste volgorde uit?
    pubid2authorid[pub_id].append(author_id)

In [5]:
authors = pd.read_csv(f"{dump_dir}/authors.csv", header=0)
authors.sample(10)

,id,ppn,is_alias,original_author_id,standardname,fullname,year,summary,creator,creation_date,modifier,modification_date,canonical_url
57101,72922,NaN,f,NaN,"Vanhauwaert, Maud",NaN,15.01.1984-,"auteur van poëzie, tekstperformer",19,"2015-06-04, 13:15:31",19.0,"2015-06-04, 13:22:41",vanhauwaert_maud
20610,24451,074506129,f,NaN,"Jansen, Margo G. H.","Jansen, Margo G. H.",1944-,NaN,1,NaN,NaN,NaN,jansen_margo_g_h
19085,41790,188861920,f,NaN,"Houtermans, Muriel","Houtermans, Muriel",NaN,NaN,1,NaN,NaN,NaN,houtermans_muriel
2219,5931,068909691,f,NaN,"Bobková, Hana","Bobková, Hana",1941-,NaN,1,NaN,NaN,NaN,bobkov_hana
4486,13782,071108602,f,NaN,"Bons, Jan","Bons, Jan",1918-,"schilder, beeldend kunstenaar",1,NaN,NaN,NaN,bons_jan
23556,35242,125345089,f,NaN,"Keizer, Bert","Keizer, A. A.",1947-,NaN,1,NaN,NaN,NaN,keizer_a_a
35247,46487,30415444X,f,NaN,"Poss, Michaela","Poss, Michaela",NaN,NaN,1,NaN,NaN,NaN,poss_michaela
37309,37782,150670672,f,NaN,"Rogiest, Peter","Rogiest, Peter",1971-,"licentiaat Germaanse talen, verantwoordelijke ...",1,NaN,19.0,"2020-02-14, 10:26:36",rogiest_peter
18991,17317,072434627,f,NaN,"Houba, Mathijs Karel Hubert","Houba, Mathijs Karel Hubert",1846-1940,NaN,1,NaN,NaN,NaN,houba_mathijs_karel_hubert
28953,20735,073501301,f,NaN,"Maidani-Gérard, Jean Pierre","Maidani-Gérard, Jean Pierre",NaN,NaN,1,NaN,NaN,NaN,maidani-g_rard_jean_pierre


In [6]:
authorid2author = dict(zip(authors['id'], authors['standardname']))

### Keywords

In [7]:
publ_keywords = pd.read_csv(f"{dump_dir}/publications_keywords.csv", header=0)
publ_keywords.sample(10)

,publication_id,keyword_id,sequence
566407,4801,46931,1260183
202864,116984,4515,275098
69012,38097,47101,91633
568984,228898,47062,799537
98180,87530,46900,1366569
406255,248505,11303,1214028
626888,235932,47191,896033
309054,255537,2155,420106
450076,3266,46933,613018
231654,134544,47174,314233


In [8]:
pubid2keywordid = defaultdict(list)
for pub_id, keyword_id in zip(publ_keywords['publication_id'], publ_keywords['keyword_id']):
    pubid2keywordid[pub_id].append(keyword_id)

In [9]:
keywords = pd.read_csv(f"{dump_dir}/keywords.csv", header=0)
keywords.sample(10)

,id,headword,summary,variants,author_id,creator,type,ppn,kmc260,kmc265,creation_date,modifier,modification_date,canonical_url
21932,18703,Mediaeval manuscripts from the Low Countries i...,(1971) 1; niet verder verschenen.,NaN,NaN,1,zaak,076718522,NaN,NaN,NaN,NaN,NaN,mediaeval_manuscripts_from_the_low_countries_i...
42864,50647,thema Vlaamse landschap,NaN,NaN,NaN,18,zaak,NaN,NaN,NaN,"2010-06-29, 10:44:51",18.0,"2010-06-29, 10:45:07",thema_vlaamse_landschap
14797,1572,historielied,liederen die betrekking hebben op historische ...,NaN,NaN,1,zaak,076529061,NaN,NaN,NaN,NaN,NaN,historielied
5468,28702,NaN,NaN,NaN,8970.0,1,auteur,085914541,NaN,NaN,NaN,NaN,NaN,burger_d
27892,23392,probleemgestuurd onderwijs,NaN,NaN,NaN,1,zaak,076769305,NaN,NaN,NaN,NaN,NaN,probleemgestuurd_onderwijs
6569,51331,Dovie,NaN,NaN,NaN,19,zaak,NaN,NaN,NaN,"2011-10-28, 14:08:30",19.0,"2011-10-28, 14:08:32",dovie
46974,53259,"Windt, Jan de","07.06.1876-09.08.1898; geoloog, ijverde voor h...",NaN,NaN,19,zaak,NaN,NaN,NaN,"2017-01-30, 10:25:48",19.0,"2017-01-30, 10:33:37",windt_jan_de
25671,47017,"over één werk - invloed, literaire betrekkinge...",inserted by new_migration,NaN,NaN,1,zaak,077049810,NaN,NaN,NaN,NaN,NaN,over_n_werk_-_invloed_literaire_betrekkingen_e...
16556,39566,izabee,NaN,NaN,NaN,1,zaak,155586157,NaN,NaN,NaN,NaN,NaN,izabee
24561,42143,NaN,NaN,NaN,23864.0,1,auteur,184827647,NaN,NaN,NaN,NaN,NaN,noord_gertjan_van


In [10]:
keywordid2keyword = {}

for kw_id, headword, auth_id, type_ in zip(keywords['id'], keywords['headword'], keywords['author_id'], keywords['type']):
    if type_ == 'auteur':
        keywordid2keyword[kw_id] = authorid2author[auth_id] + ' (auteur)'
    else: 
        keywordid2keyword[kw_id] = headword

### Publications

In [11]:
import numpy as np
publications = pd.read_csv(f"{dump_dir}/publications.csv", header=0, parse_dates=['creation_date', 'modification_date'])

In [12]:
publications['id'] = publications['id'].fillna('')
publications = publications.set_index('id')

In [13]:
author_names, keyword_names = [], []

for pub_id in publications.index:
    ans, kns = [], []

    try:
        for author_id in pubid2authorid[pub_id]:
            ans.append(authorid2author[author_id])
    except KeyError:
        pass

    try:
        for keyword_id in pubid2keywordid[pub_id]:
            kns.append(keywordid2keyword[keyword_id])
    except KeyError:
        pass

    author_names.append(ans)
    keyword_names.append(kns)

publications['authors'] = author_names
publications['keywords'] = keyword_names

In [14]:
# _ in new column names indicates: mapping not available
bntl2ris = {
            'id': 'id',
            'ppn': 'ppn_',
            'type': 'type_of_reference',
            'jaar': 'year',
            'title_description': 'title',
            'abstract': 'abstract', # geen echt abstract, maar meer informatie
            'url': 'urls',
            'containing_publication_id': 'containing_publication_id', # id van het moederrecord
            'containing_publication_ppn': '?',
            'page_numbers': 'page_numbers',
            'creator': 'creator_', # degene die record heeft aangemaakt: gaat verloren
            'creation_date': 'access_date',
            'modifier': '?', # degend die record het laatste heeft aangepast: gaat verloren
            'modification_date': 'modification_date_', # wanneer record het laatste is aangepast: gaat verloren
            'canonical_url': 'canonical_url_', # deellink naar record op BNTL-sire: gaat verloren
          }

publications = publications.rename(mapper=bntl2ris, axis=1)

In [15]:
# for dev purposes:
clean_publications = publications[[c for c in publications.columns if not (c.endswith('_') or c == '?')]].copy()
#clean_publications = clean_publications.sample(1000)
clean_publications

,type_of_reference,year,title,abstract,urls,containing_publication_id,page_numbers,access_date,authors,keywords
id,,,,,,,,,,
254444,artikeltijdschrift,2009,"Campenhout, Frans van. Ropsiennes, de nieuwe d...","Over: Meekers, Mark. Ropsiennes: gedichten bij...",NaN,NaN,NaN,2009-06-26 11:32:26,"[Campenhout, Frans van]","[Meekers, Mark (auteur)]"
254442,artikeltijdschrift,2009,"Campenhout, Frans van. Dichteres en romancière...",NaN,NaN,NaN,NaN,2009-06-26 11:27:36,"[Campenhout, Frans van]","[Bruylants, Liane (auteur)]"
313784,artikeltijdschrift,2018,"Lievois, Katrien, Hanne Kloots & Noureddine Na...",P. 46 Noten; p. 46 Bibliografie; p. 47 Bijlage...,NaN,NaN,NaN,2023-11-29 16:36:19,"[Lievois, Katrien, Kloots, Hanne, Nahed, Noure...","[literair vertalen, vertalingen uit het Frans,..."
171436,artikeltijdschrift,1965,Interprovinciale Prijs voor monografie aan een...,"N.a.v.: Deblaere, Albert. De mystieke schrijfs...",NaN,166925.0,"vol. 14 (1965), afl. 79 (jan-feb), p. 66-67.",NaT,[n.v.t.],"[Deblaere, Albert (auteur), Petyt, Maria (aute..."
246627,artikelboek,2004,"Willaert, Frank. Margaret's booklets: memory i...","Lezing, gehouden op het congres ""Memory in the...",NaN,NaN,NaN,2008-06-18 12:05:19,"[Willaert, Frank]","[Ruusbroec, Jan van (auteur), memoria, tekstin..."
...,...,...,...,...,...,...,...,...,...,...
254432,recensie,1962,"Plard, Henri. Une mystique flamande retrouvée:...",NaN,NaN,NaN,NaN,2009-06-26 10:31:29,"[Plard, Henri]",[]
129705,recensie,2003,"Offermans, Cyrille. \r\nIn: Ons erfdeel (Rekk...",NaN,NaN,NaN,NaN,NaT,"[Offermans, Cyrille]",[]
129707,boek,2003,Nederlands-Afghaanse grammatica = Nīdarlandī-P...,P. 181-182 Literatuur.\n1e-2e dr. ?,NaN,NaN,NaN,NaT,"[Nasir Ahmad, Sayed]","[leerboeken voor buitenlanders, Afghaansspreke..."


In [16]:
# Naar: https://en.wikipedia.org/wiki/RIS_(file_format)#Type_of_reference
map_type = {'artikeltijdschrift': 'JOUR', # Journal/periodical (article)
            'recensie': 'RPRT', # Report -> Temporary; will be mapped to JOUR below
            'artikelboek': 'CHAP', # Book section/chapter
            'boek': 'BOOK', # Book (whole)
            'website': 'WEB', # Web page
            'artikelwebsite': 'EJOUR', # suggereert tijdschrift?
            'specialetijdschriftaflevering': 'JFULL', # Journal/periodical (full) -> niet speciaal aan te geven?
            'tijdschriftofreeks': 'JFULL', # Journal/periodical (full)
            'cddvd': 'ADVS', # Audiovisual material
           }

In [17]:
clean_publications['type_of_reference'] = clean_publications['type_of_reference'].replace(map_type)

clean_publications['abstract'] = clean_publications['abstract'].fillna('').str.strip()

clean_publications['start_page'] = clean_publications['page_numbers'].fillna('').str.strip()

clean_publications['year'] = clean_publications['year'].fillna('').str.strip()

clean_publications['access_date'] = clean_publications['access_date'].dt.strftime('%d-%m-%Y').fillna('').str.strip()

clean_publications['urls'] = clean_publications['urls'].fillna('').str.strip()
clean_publications['urls'] = [[u] if u else [] for u in clean_publications['urls']]

# same for all
clean_publications['name_of_database'] = 'Bibliografie van de Nederlandse Taal- en Literatuurwetenschap (BNTL)'

### URLs

URLs have been added in an additional table too.

In [18]:
publ_urls = pd.read_csv(f"{dump_dir}/urls.csv", header=0)
publ_urls.sample(10)

,publication_id,url
5324,134964,http://www.dbnl.org/tekst/will001inle01_01/
5469,293213,http://www.dereactor.org/home/detail/hij_doet_...
5251,298058,https://doi.org/10.5117/TVGESCH2012.2.B10
3771,279960,http://muse.jhu.edu.access.authkb.kb.nl/journa...
6544,269790,http://www.dbnl.org/tekst/_med003mede01_01/_me...
4755,277811,https://doi.org/10.5117/IVN2012.0.PIET
11844,98897,https://www.dbnl.org/tekst/_taa014199701_01/_t...
7686,302781,https://www.dereactor.org/teksten/schrijven-al...
1941,300247,https://www.elkedagboeken.be/dw-b/literaire-kr...
9863,315704,https://lup.be/products/177206


In [19]:
pubid2url = defaultdict(list)
for pub_id, url in zip(publ_urls['publication_id'], publ_urls['url']):
    pubid2url[pub_id].append(url)

In [20]:
clean_publications.head(5)

,type_of_reference,year,title,abstract,urls,containing_publication_id,page_numbers,access_date,authors,keywords,start_page,name_of_database
id,,,,,,,,,,,,
254444,JOUR,2009,"Campenhout, Frans van. Ropsiennes, de nieuwe d...","Over: Meekers, Mark. Ropsiennes: gedichten bij...",[],NaN,NaN,26-06-2009,"[Campenhout, Frans van]","[Meekers, Mark (auteur)]",,Bibliografie van de Nederlandse Taal- en Liter...
254442,JOUR,2009,"Campenhout, Frans van. Dichteres en romancière...",,[],NaN,NaN,26-06-2009,"[Campenhout, Frans van]","[Bruylants, Liane (auteur)]",,Bibliografie van de Nederlandse Taal- en Liter...
313784,JOUR,2018,"Lievois, Katrien, Hanne Kloots & Noureddine Na...",P. 46 Noten; p. 46 Bibliografie; p. 47 Bijlage...,[],NaN,NaN,29-11-2023,"[Lievois, Katrien, Kloots, Hanne, Nahed, Noure...","[literair vertalen, vertalingen uit het Frans,...",,Bibliografie van de Nederlandse Taal- en Liter...
171436,JOUR,1965,Interprovinciale Prijs voor monografie aan een...,"N.a.v.: Deblaere, Albert. De mystieke schrijfs...",[],166925.0,"vol. 14 (1965), afl. 79 (jan-feb), p. 66-67.",,[n.v.t.],"[Deblaere, Albert (auteur), Petyt, Maria (aute...","vol. 14 (1965), afl. 79 (jan-feb), p. 66-67.",Bibliografie van de Nederlandse Taal- en Liter...
246627,CHAP,2004,"Willaert, Frank. Margaret's booklets: memory i...","Lezing, gehouden op het congres ""Memory in the...",[],NaN,NaN,18-06-2008,"[Willaert, Frank]","[Ruusbroec, Jan van (auteur), memoria, tekstin...",,Bibliografie van de Nederlandse Taal- en Liter...


In [21]:
more_urls = []

for pub_id, pub_row in clean_publications.iterrows():
    ur = pub_row['urls']
    ur.extend(pubid2url.get(pub_id, []))
    more_urls.append(ur)

clean_publications['urls'] = more_urls



### Mother references

In [22]:
pd.set_option('display.max_colwidth', None)

In [23]:
secondary_titles = []

for idx, row in clean_publications.iterrows():
    if not math.isnan(row['containing_publication_id']):
        jtitle = publications.loc[int(row['containing_publication_id'])]['title']
        secondary_titles.append(jtitle)
    else:
        secondary_titles.append('')

clean_publications['secondary_title'] = secondary_titles

In [24]:
clean_publications = clean_publications[[c for c in clean_publications.columns \
                                         if c not in ('containing_publication_id', 'page_numbers')]].copy()

Remove literary authors mentioned as publication authors:

In [25]:
clean_authors = []

for authors in clean_publications['authors']:
    if len(authors) == 1 and authors[0].lower() == 'n.v.t.':
        clean_authors.append([])
    else:
        clean_authors.append(authors)

clean_publications['authors'] = clean_authors

In [26]:
clean_authors = []

for authors, keywords in zip(clean_publications['authors'], clean_publications['keywords']):
    new_authors = authors.copy()
    if authors and keywords:
        authors_mentioned = set()
        for kw in keywords:
            if '(auteur)' in kw:
                authors_mentioned.add(kw.strip().replace(' (auteur)', '').strip())
        new_authors = [a for a in authors if a in set(authors) - authors_mentioned]
    clean_authors.append(new_authors)

clean_publications['authors'] = clean_authors


In [27]:
clean_publications

,type_of_reference,year,title,abstract,urls,access_date,authors,keywords,start_page,name_of_database,secondary_title
id,,,,,,,,,,,
254444,JOUR,2009,"Campenhout, Frans van. Ropsiennes, de nieuwe dichtbundel van Mark Meekers.\r\nIn: Verba (Brussel): 13 (2009) 2 (mei-juli) 14.","Over: Meekers, Mark. Ropsiennes: gedichten bij het leven en werk van Félicien Rops (1833-1898). Leuven: P, 2009.",[],26-06-2009,"[Campenhout, Frans van]","[Meekers, Mark (auteur)]",,Bibliografie van de Nederlandse Taal- en Literatuurwetenschap (BNTL),
254442,JOUR,2009,"Campenhout, Frans van. Dichteres en romancière Liane Bruylants overleden.\r\nIn: Verba (Brussel): 13 (2009) 2 (mei-juli) 5-8.",,[],26-06-2009,"[Campenhout, Frans van]","[Bruylants, Liane (auteur)]",,Bibliografie van de Nederlandse Taal- en Literatuurwetenschap (BNTL),
313784,JOUR,2018,"Lievois, Katrien, Hanne Kloots & Noureddine Nahed. Kif-kif?: Arabische ontleningen in de Nederlandse, Spaanse en Arabische versie van ""Kiffe kiffe demain"".\r\nIn: Meertaligheid. Speciaal gedeelte van: Filter (Nijmegen): 25 (2018) 3 (sep) 37-47.","P. 46 Noten; p. 46 Bibliografie; p. 47 Bijlage: vertalingen van ""Kiffe kiffe demain"".",[https://www.tijdschrift-filter.nl/jaargangen/2018/253/kif-kif-37-47/],29-11-2023,"[Lievois, Katrien, Kloots, Hanne, Nahed, Noureddine]","[literair vertalen, vertalingen uit het Frans, meertaligheid, multiculturaliteit, Arabische leenwoorden, straattaal]",,Bibliografie van de Nederlandse Taal- en Literatuurwetenschap (BNTL),
171436,JOUR,1965,"Interprovinciale Prijs voor monografie aan een Westvlaming, [door] A.S.\r\nIn: West-Vlaanderen: 14 (1965) 79 (jan-feb) 66-67.","N.a.v.: Deblaere, Albert. De mystieke schrijfster Maria Petyt, (1623-1677). Gent: Secretarie der Academie, 1962.",[],,[],"[Deblaere, Albert (auteur), Petyt, Maria (auteur)]","vol. 14 (1965), afl. 79 (jan-feb), p. 66-67.",Bibliografie van de Nederlandse Taal- en Literatuurwetenschap (BNTL),"West-Vlaanderen: tweemaandelijks tijdschrift voor kunst en cultuur; uitgegeven door het Christelijk Vlaams Kunstenaars Verbond. St. Michiels-Brugge: Christelijk Vlaams Kunstenaarsverbond, 1950-1965."
246627,CHAP,2004,"Willaert, Frank. Margaret's booklets: memory in ""Vanden seven sloten"" by Jan van Ruusbroec.\r\nIn: Medieval memory: image and text; edited by Frank Willaert, Herman Braet, Tom Mertens, Theofiel Venckeleer. Turnhout: Brepols, 2004, p. 99-128.\r\n(Textes et études du Moyen Âge; 27).","Lezing, gehouden op het congres ""Memory in the Middle Ages"" aan de Universiteit Antwerpen, 7-9 maart 2002.",[],18-06-2008,"[Willaert, Frank]","[Ruusbroec, Jan van (auteur), memoria, tekstinterpretatie]",,Bibliografie van de Nederlandse Taal- en Literatuurwetenschap (BNTL),
...,...,...,...,...,...,...,...,...,...,...,...
254432,RPRT,1962,"Plard, Henri. Une mystique flamande retrouvée: Maria Petyt (1623-1677).\r\nIn: Études germaniques (Paris): 17 (1962) 3 (juill-sep) 347-354.",,[],26-06-2009,"[Plard, Henri]",[],,Bibliografie van de Nederlandse Taal- en Literatuurwetenschap (BNTL),
129705,RPRT,2003,"Offermans, Cyrille. \r\nIn: Ons erfdeel (Rekkem): 46 (2003) 3 (juni) 446-448.",,[],,"[Offermans, Cyrille]",[],,Bibliografie van de Nederlandse Taal- en Literatuurwetenschap (BNTL),
129707,BOOK,2003,"Nederlands-Afghaanse grammatica = Nīdarlandī-Paśtū paśwayah / Sayed Nasir Ahmad (Sayyid Naṣīr Aḥmad). - 3e, herz. dr. - Utrecht : Nederlands Centrum Buitenlanders, cop. 2003. - XIV, 182 p. ; 24 cm Tekst in het Nederlands en het Pasjtoe. - Ook o.d.t.: Nederlands-Pashto grammatica. - Met lit. opg. ISBN 90-5517-169-7.",P. 181-182 Literatuur.\n1e-2e dr. ?,[],,"[Nasir Ahmad, Sayed]","[leerboeken voor buitenlanders, Afghaanssprekenden, grammatica]",,Bibliografie van de Nederlandse Taal- en Literatuurwetenschap (BNTL),


Reviews become journal articles, but get a keyword:

In [28]:
types, keywords = [], []

for idx, row in clean_publications.iterrows():
    ty, kws = row['type_of_reference'], row['keywords']
    if ty == 'RPRT':
        ty = 'JOUR'
        kws.append('recensie')

    types.append(ty)
    keywords.append(kws)


clean_publications['type_of_reference'] = types
clean_publications['keywords'] = keywords

### Add short journal titles

Load manually corrected sample:

In [29]:
journal_titles = pd.read_excel('../data/journal_titles_master.xlsx')
journal_titles['normalized'] = journal_titles['normalized'].str.strip()
journal_titles['secondary_title'] = journal_titles['secondary_title'].str.strip()
journal_titles['issn'] = journal_titles['issn'].str.strip()
journal_lookup = {}
for long, short, issn in zip(journal_titles['secondary_title'], journal_titles['normalized'], journal_titles['issn']):
    journal_lookup[' '.join(long.strip().split()).replace("\r\n", "\n")] = (short, issn)

In [30]:
journal_titles

,secondary_title,normalized,count,issn
0,"Ons erfdeel: kultureel tijdschrift voor Zuidvlaamse werking in Vlaams-Nederlands-Suidafrikaanse samenwerking. Menen: Deleu, 1957-...",Ons erfdeel,2660.0,0030-2651
1,"Dietsche warande en Belfort: tijdschrift voor letterkunde, kunst en geestesleven. Gent: Siffer; Haarlem: Coebergh; Antwerpen [etc.]: Standaard, 1900-....",Dietsche warande en Belfort,2461.0,0012-2645
2,"De nieuwe taalgids: tweemaandelijks tijdschrift. Groningen: Tjeenk Willink, 1907-1995.",De nieuwe taalgids,2359.0,0028-9922
3,"Bzzlletin; Stichting BZZTôH Teater. Voorburg: Stichting BZZTôH, 1972-2004.",Bzzlletin,1638.0,0165-0858
4,"Poëziekrant: tweemaandelijks tijdschrift. Gent: Poëziecentrum, 1976-...",Poëziekrant,1573.0,2030-0638
...,...,...,...,...
4202,"De schalm : tijdschrift voor buurt- en buurthuiswerk / uitg. van de Nederlandse Bond van Volkshuizen. - Jrg. 1 (1935/36) - jrg. 27, nr. 6 (1970). - Utrecht : Nederlandse Bond van Volkshuizen [etc.], 1935-1970. - 19-23 cm Verscheen 1x per 2 maanden. - Ondertitel varieert. - Corporatie later o.d.n.: Nederlandse Bond voor Sociaal-Cultureel Vormingswerk. - Als interim-nummer verscheen in aug. 1971: Missing link. - Voortgez. als: Attak = ISSN 0044-9962. - Met: De Schalm bibliografie, 1935-1970, 27 jaargangen. - 1972.",De schalm,1.0,NaN
4203,"27 Squibs; [redactie: G. E. Booij... et al.]. Groningen: Wolters-Noordhoff, 1977. 116 p. (p. 377-492).\n(Speciaal nummer van: Spektator; jrg. 6, nr. 7-8, (feb-mrt)).",Spektator,1.0,NaN
4204,"The journal of ecclesiastical history. London: Faber and Faber, 1950-....",The journal of ecclesiastical history,1.0,0022-0469
4205,"Studia missionalia; Pontificia Universitas Gregoriana. Facultas Missiologica. Roma: Università Gregoriana Editrice, 1943-....",Studia missionalia,1.0,0080-3987


In [31]:
from collections import Counter
miss_counter = Counter()

secondary_titles, issns = [], []
for _, publ in clean_publications.iterrows():
    short, issn = publ['secondary_title'], None
    if publ['type_of_reference'] in {'JOUR', 'JFULL'} and publ['secondary_title'].strip():
        t = ' '.join(publ['secondary_title'].strip().split()).replace("\r\n", "\n")
        try:
            short, issn = journal_lookup[t]
        except KeyError:
            try:
                t = t.split('\n')[0].strip()
                short, issn = journal_lookup[t]
            except KeyError:
                miss_counter[publ['secondary_title']] += 1
    
    secondary_titles.append(short)
    issns.append(issn)

clean_publications['secondary_title'] = secondary_titles
clean_publications['issn'] = issns

for k in miss_counter:
    print(k)

Mainzer Beobachter. Amsterdam: Aarts, 1987.
T'Sjoen, Yves. Stem en tegenstem: over poëzie en poëtica: dubbelessays over hedendaagse Nederlandse poëzie. Amsterdam [etc.]: Atlas, 2004. 260 p.
Hugo Claus, het toneel en Vlaanderen / Freddy de Vree.
In: Hugo Claus, het toneel en Vlaanderen / Freddy de Vree: vol. 47 (1971 [versch. 1970]), pag. 51-59.
Naar morgen. Eindhoven: Opwenteling, 1969-1991. 
(Opwenteling).
In honour of Anton Reichling on the occasion of his seventieth birthday; ed. by W. Sidney Allen, Simon C. Dik, J. G. Kooij [et al.]. Amsterdam: North-Holland, 1968. VIII, 637 p. 
(Speciaal nr. van: Lingua (Haarlem): 21 (1968)).
Over Willem Elsschot: beschouwingen en interviews; red. Annemarie Kets-Vree. 's-Gravenhage: BZZTôH, 1982. 293 p. 
(BZZTôH literair archief).


In [32]:
clean_publications.sample(50)

,type_of_reference,year,title,abstract,urls,access_date,authors,keywords,start_page,name_of_database,secondary_title,issn
id,,,,,,,,,,,,
268107,CHAP,2009,"Vandermeersch, Guido. Maria Petyt, Zuid-Vlaamse mystica.\r\nIn: De Zuidelijke Nederlanden; Kristof Papin, Luc Rochtus, Johan van Herreweghe (ed.); met bijdragen van Jozef Bossu ... [et al.]. [Antwerpen]: Marnixring Lieven Gevaert, 2009, p. 267.",,[],23-03-2011,"[Vandermeersch, Guido]","[Petyt, Maria (auteur)]",,Bibliografie van de Nederlandse Taal- en Literatuurwetenschap (BNTL),,None
109668,JOUR,1996-1997,"De fijne kneepjes, 1-3 / Silvia Kooij, Annemiek Manuel.\nIn: Van Dale taalbrief: vol. 5 (1996), afl. 12, pag. 1-2; vol. 6 (1997), afl. 3, pag. 4; afl. 7-8, pag. 6.","Bevat interviews met Annemiek Manuel (5 (1996) 12, 1-2), Paul Spendel (6 (1997) 3, 4) en Emmy Nieman (6 (1997) 7-8, 6).",[],,"[Kooij, Silvia, Verheul, Ingeborg]","[normatieve studies, zakelijke tekst]","vol. 6 (1997), afl. 7-8 p. 6.",Bibliografie van de Nederlandse Taal- en Literatuurwetenschap (BNTL),Van Dale taalbrief,0927-3352
78085,JOUR,1994,"Een dikke sterke monnik / Frédéric Bastet.\nIn: Origine Haarlem: (1994), afl. 1 (feb), pag. 9.",,[],,"[Bastet, F. L.]","[literatuurwetenschap: per auteur - vanaf 1830 - descriptieve studies, algemeen biografisch: persoonlijkheid, overige activiteiten, Vosmaer, C. (auteur), beeldende kunst]","(1994), afl. 1 (feb), p. 9.",Bibliografie van de Nederlandse Taal- en Literatuurwetenschap (BNTL),Origine,NaN
180237,CHAP,1966,"Meermans, L. Aantekeningen van een fysicus bij het lezen van Achterberg. \r\nIn: Nieuw kommentaar op Achterberg; samengesteld door Bert Bakker en Andries Middeldorp. Den Haag: Bert Bakker/Daamen, 1966, p. 163-174.",,[],,"[Meermans, L.]","[Achterberg, Gerrit (auteur)]",p. 163-174.,Bibliografie van de Nederlandse Taal- en Literatuurwetenschap (BNTL),,None
176941,JOUR,1966,"Discriminatie van lettervormen bij kinderen met lees- en schrijfstoornissen.\nIn: Ned. tijdschr. psychol.: vol. 21 (1966), pag. 194-210.",Blz. 208-209 Samenvatting - summary,[],,"[Sanders, C. H., Meel, J. M. van]","[taalwetenschap: descriptieve studies - psycholinguïstiek - studies betreffende niet normaal taalgebruik, leesstoornis, schrijfstoornis]","vol. 21 (1966), p. 194-210.",Bibliografie van de Nederlandse Taal- en Literatuurwetenschap (BNTL),Nederlandsch tijdschrift voor de psychologie,0028-2235
127635,JOUR,1949,"Eckeren, Gerard van. ""Downs"" en ""ups"" in het werk van Jo Boer. \r\nIn: Het boek van nu (Baarn): 2 (1948-1949) 10 (juni 1949) 189-190.","Over: Boer, Jo. Beeld en spiegelbeeld. Amsterdam: De Muiderkring, 1948; en: Boer, Jo. Kruis of munt: roman. 's-Gravenhage: A. A. M. Stols, 1949.",[],,"[Eckeren, Gerard van]","[Boer, Jo (auteur)]","vol. 2 (1948-1949), afl. 10 (jun), p. 189-190.",Bibliografie van de Nederlandse Taal- en Literatuurwetenschap (BNTL),Het boek van nu,NaN
235766,JOUR,1983,"De Baandrekel, een neie Draense verhaolenbundel.\nIn: Oeze volk Stadskanaal: vol. 27 (1983), afl. 2 (feb), pag. 29-30.",,[],,"[Hadderingh, H.]","[literatuurwetenschap: beperkt in de ruimte - bepaalde gebieden binnen het Nederlandse taalgebied - vanaf 1830 - descriptieve studies, Drente, Poortman, J. (auteur), streektaalliteratuur]","vol. 27 (1983), afl. 2 (feb), p. 29-30.",Bibliografie van de Nederlandse Taal- en Literatuurwetenschap (BNTL),Oeze volk,1381-7779
136008,JOUR,2002,"[Over: Platteau, Pierre. Rue Bonnevie. Amsterdam [etc.], 2002] / Jooris van Hulle.\nIn: Leesidee Antwerp.: vol. 8 (2002), afl. 6 (sep), pag. 431.",,[],,"[Hulle, Jooris van]","[literatuurwetenschap: per auteur - vanaf 1830 - descriptieve studies, besprekingsartikelen en recensies over één werk, Platteau, Pierre (auteur)]","vol. 8 (2002), afl. 6 (sep), p. 431.",Bibliografie van de Nederlandse Taal- en Literatuurwetenschap (BNTL),Leesidee,1370-1053
312252,JOUR,2022,"Andries, Fien, Geert Brône, Myriam Vermeerbergen. Stance in Flemish Sign Language: a multimodal and polysemiotic phenomenon.\r\nIn: On the semiotic diversit

In [33]:
clean_publications[clean_publications['title'].str.contains('Renate Rubinstein: an introduction')]

,type_of_reference,year,title,abstract,urls,access_date,authors,keywords,start_page,name_of_database,secondary_title,issn
id,,,,,,,,,,,,
317309,JOUR,1987,"Joldersma, Hermina. Renate Rubinstein: an introduction.\r\nIn: Canadian journal of Netherlandic studies: 8 (1987) 1 (Spring) 40-51.","P. 50-51 Works cited.\r\n\r\nLezing, gehouden in het kader van het Holland in Canada festival op de McMaster University, mei 1987 te Hamilton (Ontario).\r\n\r\nOok in: Voorwaarts / Forward: an anthology of writing from the Canadian Journal of Netherlandic studies, on the occasion of the 25th anniversary of the Canadian Association for the Advancement of Netherlandic Studies (1971-1996); ed. by: Augustinus P. Dierick, Basil Kingstone. Speciaal nr. van: Canadian Journal of Netherlandic Studies.: 17 (1996) 1-2, 190-199.",[https://caans-acaen.ca/Journal/issues_online/Issue_VIII_i_1987/Joldersma.pdf],23-01-2024,"[Joldersma, Hermina]","[Rubinstein, Renate (auteur)]",,Bibliografie van de Nederlandse Taal- en Literatuurwetenschap (BNTL),,None
317500,JOUR,1996,"Joldersma, Hermina. Renate Rubinstein: an introduction.\r\nIn: Voorwaarts/Forward: an anthology of writing from the Canadian Journal of Netherlandic Studies; [gastredacteurs: Augustinus P. Dierick en Basil Kingstone]. Speciaal nummer van: Canadian journal of Netherlandic studies (Windsor): 17 (1996) 1-2 (Spring-Fall) 190-199.","P. 199 Works cited.\r\n\r\nLezing, gehouden in het kader van het Holland in Canada festival op de McMaster University, mei 1987 te Hamilton (Ontario).\r\n\r\nOok in eerder nummer verschenen als: Joldersma, Hermina. Renate Rubinstein: an introduction.\r\nIn: Canadian journal of Netherlandic studies: 8 (1987) 1 (Spring) 40-51.",[https://caans-acaen.ca/Journal/issues_online/Issue_XVII_i_ii_1996/JOLDERSMA.pdf],05-02-2024,"[Joldersma, Hermina]","[Rubinstein, Renate (auteur)]",,Bibliografie van de Nederlandse Taal- en Literatuurwetenschap (BNTL),,None


In [34]:
from rispy import LIST_TYPE_TAGS
print(LIST_TYPE_TAGS)

['A1', 'A2', 'A3', 'A4', 'AU', 'KW', 'N1', 'UR']


In [35]:
clean_publications[clean_publications['urls'] != '']

,type_of_reference,year,title,abstract,urls,access_date,authors,keywords,start_page,name_of_database,secondary_title,issn
id,,,,,,,,,,,,
254444,JOUR,2009,"Campenhout, Frans van. Ropsiennes, de nieuwe dichtbundel van Mark Meekers.\r\nIn: Verba (Brussel): 13 (2009) 2 (mei-juli) 14.","Over: Meekers, Mark. Ropsiennes: gedichten bij het leven en werk van Félicien Rops (1833-1898). Leuven: P, 2009.",[],26-06-2009,"[Campenhout, Frans van]","[Meekers, Mark (auteur)]",,Bibliografie van de Nederlandse Taal- en Literatuurwetenschap (BNTL),,None
254442,JOUR,2009,"Campenhout, Frans van. Dichteres en romancière Liane Bruylants overleden.\r\nIn: Verba (Brussel): 13 (2009) 2 (mei-juli) 5-8.",,[],26-06-2009,"[Campenhout, Frans van]","[Bruylants, Liane (auteur)]",,Bibliografie van de Nederlandse Taal- en Literatuurwetenschap (BNTL),,None
313784,JOUR,2018,"Lievois, Katrien, Hanne Kloots & Noureddine Nahed. Kif-kif?: Arabische ontleningen in de Nederlandse, Spaanse en Arabische versie van ""Kiffe kiffe demain"".\r\nIn: Meertaligheid. Speciaal gedeelte van: Filter (Nijmegen): 25 (2018) 3 (sep) 37-47.","P. 46 Noten; p. 46 Bibliografie; p. 47 Bijlage: vertalingen van ""Kiffe kiffe demain"".",[https://www.tijdschrift-filter.nl/jaargangen/2018/253/kif-kif-37-47/],29-11-2023,"[Lievois, Katrien, Kloots, Hanne, Nahed, Noureddine]","[literair vertalen, vertalingen uit het Frans, meertaligheid, multiculturaliteit, Arabische leenwoorden, straattaal]",,Bibliografie van de Nederlandse Taal- en Literatuurwetenschap (BNTL),,None
171436,JOUR,1965,"Interprovinciale Prijs voor monografie aan een Westvlaming, [door] A.S.\r\nIn: West-Vlaanderen: 14 (1965) 79 (jan-feb) 66-67.","N.a.v.: Deblaere, Albert. De mystieke schrijfster Maria Petyt, (1623-1677). Gent: Secretarie der Academie, 1962.",[],,[],"[Deblaere, Albert (auteur), Petyt, Maria (auteur)]","vol. 14 (1965), afl. 79 (jan-feb), p. 66-67.",Bibliografie van de Nederlandse Taal- en Literatuurwetenschap (BNTL),West-Vlaanderen,NaN
246627,CHAP,2004,"Willaert, Frank. Margaret's booklets: memory in ""Vanden seven sloten"" by Jan van Ruusbroec.\r\nIn: Medieval memory: image and text; edited by Frank Willaert, Herman Braet, Tom Mertens, Theofiel Venckeleer. Turnhout: Brepols, 2004, p. 99-128.\r\n(Textes et études du Moyen Âge; 27).","Lezing, gehouden op het congres ""Memory in the Middle Ages"" aan de Universiteit Antwerpen, 7-9 maart 2002.",[],18-06-2008,"[Willaert, Frank]","[Ruusbroec, Jan van (auteur), memoria, tekstinterpretatie]",,Bibliografie van de Nederlandse Taal- en Literatuurwetenschap (BNTL),,None
...,...,...,...,...,...,...,...,...,...,...,...,...
254432,JOUR,1962,"Plard, Henri. Une mystique flamande retrouvée: Maria Petyt (1623-1677).\r\nIn: Études germaniques (Paris): 17 (1962) 3 (juill-sep) 347-354.",,[],26-06-2009,"[Plard, Henri]",[recensie],,Bibliografie van de Nederlandse Taal- en Literatuurwetenschap (BNTL),,None
129705,JOUR,2003,"Offermans, Cyrille. \r\nIn: Ons erfdeel (Rekkem): 46 (2003) 3 (juni) 446-448.",,[],,"[Offermans, Cyrille]",[recensie],,Bibliografie van de Nederlandse Taal- en Literatuurwetenschap (BNTL),,None
129707,BOOK,2003,"Nederlands-Afghaanse grammatica = Nīdarlandī-Paśtū paśwayah / Sayed Nasir Ahmad (Sayyid Naṣīr Aḥmad). - 3e, herz. dr. - Utrecht : Nederlands Centrum Buitenlanders, cop. 2003. - XIV, 182 p. ; 24 cm Tekst in het Nederlands en het Pasjtoe. - Ook o.d.t.: Nederlands-Pashto grammatica. - Met lit. opg. ISBN 90-5517-169-7.",P. 181-182 Literatuur.\n1e-2e dr. ?,[],,"[Nasir Ahmad, Sayed]","[leerboeken voor buitenlanders, Afghaanssprekenden, grammatica]",,Bibliografie van de Nederlandse Taal- en Literatuurwetenschap (BNTL),,None


## Dump to RIS

In [36]:
def dump2ris(pubs, bn):
    records = []
    for record in pubs.to_dict(orient='records'):
        records.append({k:v for k, v in record.items() if v})

    with open(f'{bn}.ris', 'w') as bibliography_file:
        rispy.dump(records, bibliography_file)
    

In [37]:
def clean_dir(dir_name):
    try:
        shutil.rmtree(dir_name)
    except FileNotFoundError:
        pass
    os.mkdir(dir_name)

dump_dir = '../data/ris-dump'
clean_dir(dump_dir)

In [38]:
def to_decade(year):
    try:
        decade = int(str(int(year))[:-1])
        if decade < 194: # lump everything pre-1945 together under misc
            decade = 'misc'
        else:
            decade = str(decade) + '0s'
    except ValueError:
        decade = 'misc'
    
    return decade

clean_publications.loc[:, 'decade'] = clean_publications['year'].apply(to_decade).copy()
clean_publications.value_counts('decade')

decade
1990s    67537
1980s    54628
2000s    46647
1970s    36057
2010s    31339
1960s    25846
1940s    13171
misc     11223
1950s     8313
2020s     8104
2030s        1
Name: count, dtype: int64

In [39]:
sizes = []
for decade, decade_pubs in clean_publications.groupby('decade'):
    clean_dir(f'{dump_dir}/{decade}')
    print(f'- {decade}')
    for ptype, ptype_pubs in decade_pubs.groupby('type_of_reference'):
        print('    +', ptype, len(ptype_pubs))
        dump2ris(ptype_pubs.drop('decade', axis=1), f'{dump_dir}/{decade}/{ptype}')
        sizes.append((len(ptype_pubs), decade, ptype))

sizes.sort(reverse=True)
print(sizes[:5])

- 1940s
    + BOOK 1442
    + CHAP 1764
    + JFULL 68
    + JOUR 9897
- 1950s
    + BOOK 917
    + CHAP 1161
    + JFULL 17
    + JOUR 6218
- 1960s
    + BOOK 2185
    + CHAP 4084
    + JFULL 190
    + JOUR 19387
- 1970s
    + BOOK 3623
    + CHAP 6650
    + JFULL 273
    + JOUR 25511
- 1980s
    + ADVS 2
    + BOOK 6722
    + CHAP 12289
    + JFULL 619
    + JOUR 34995
    + WEB 1
- 1990s
    + ADVS 33
    + BOOK 7992
    + CHAP 14247
    + EJOUR 97
    + JFULL 660
    + JOUR 44498
    + WEB 10
- 2000s
    + ADVS 54
    + BOOK 6229
    + CHAP 11027
    + EJOUR 628
    + JFULL 479
    + JOUR 27745
    + WEB 485
- 2010s
    + ADVS 9
    + BOOK 3891
    + CHAP 6300
    + EJOUR 569
    + JFULL 442
    + JOUR 20043
    + WEB 85
- 2020s
    + ADVS 2
    + BOOK 824
    + CHAP 1627
    + EJOUR 251
    + JFULL 161
    + JOUR 5238
    + WEB 1
- 2030s
    + BOOK 1
- misc
    + ADVS 3
    + BOOK 1182
    + CHAP 470
    + EJOUR 2
    + JFULL 6051
    + JOUR 3483
    + WEB 32
[(44498, '1990s', 'JO

### Journal titles

Dump table of original journal title, normalized journal title and ISSN for manual correction so that it can be re-used for mapping:

In [40]:
journal_pubs = clean_publications[clean_publications['type_of_reference'].isin({'JOUR', 'JFULL'})].copy()
# some pubs lack a secondary journal title...
journal_pubs = journal_pubs[~journal_pubs['secondary_title'].isna()]
journal_pubs = journal_pubs[journal_pubs['secondary_title'].str.strip() != '']

In [41]:
title_splitter = re.compile(r'(?<=([a-z0-9]))(\.\-*\s)|(\s\/|\;\s)')
def clean_journal_title(title):
    title = title.replace('...', '')
    clean_title = title_splitter.split(title)[0]
    clean_title = clean_title.replace(' : ', ': ').strip()
    clean_title = clean_title.split(': ')[0].strip()
    return clean_title

In [42]:
jour = journal_pubs['secondary_title'].value_counts().reset_index()
jour['normalized'] = jour['secondary_title'].apply(clean_journal_title)
jour = jour[['secondary_title', 'normalized', 'count']]

In [43]:
jour.head(30)

,secondary_title,normalized,count
0,Ons erfdeel,Ons erfdeel,2660
1,Dietsche warande en Belfort,Dietsche warande en Belfort,2461
2,De nieuwe taalgids,De nieuwe taalgids,2359
3,Bzzlletin,Bzzlletin,1638
4,Poëziekrant,Poëziekrant,1573
5,Onze taal,Onze taal,1322
6,Vlaanderen,Vlaanderen,1312
7,De gids,De gids,1249
8,Levende talen,Levende talen,1239
9,Tijdschrift voor Nederlandse taal- en letterkunde,Tijdschrift voor Nederlandse taal- en letterkunde,962


In [44]:
fjour = clean_publications[clean_publications['type_of_reference'] == 'JFULL']
fjour = fjour[fjour['authors'].str.len() == 0] # avoid special issues etc.

issn_pattern = re.compile(r'ISSN (\d{4}-\d{3}[\dX])')

def extract_issn(text):
    issns = list(re.findall(issn_pattern, text))
    if issns:
        return issns[0] # select first, if multiple mentioned
    else:
        return None
    
fjour['issn'] = fjour['abstract'].apply(extract_issn)
fjour['normalized'] = fjour['title'].apply(clean_journal_title)
fjour = fjour[['normalized', 'issn']]
fjour = fjour[~fjour['issn'].isna()]
fjour

,normalized,issn
id,,
129758,Jaarboek Jet Jorssen Genootschap,1378-0883
129767,Cultuurbericht,1872-1265
139578,Villers,1371-6360
161368,De bibliotheekgids,0006-1956
164819,Karakter,0022-8982
...,...,...
126608,Jaarboek,0770-7517
126723,Sjeppelroot thir(s)ty one,0772-8921
126904,Lyra,1570-4904


In [45]:
journal2issn = dict(zip(fjour['normalized'], fjour['issn']))

jour['issn'] = [journal2issn[nt]  if nt in journal2issn else ''\
                for nt in jour['normalized']]
jour

,secondary_title,normalized,count,issn
0,Ons erfdeel,Ons erfdeel,2660,0030-2651
1,Dietsche warande en Belfort,Dietsche warande en Belfort,2461,0012-2645
2,De nieuwe taalgids,De nieuwe taalgids,2359,0028-9922
3,Bzzlletin,Bzzlletin,1638,0165-0858
4,Poëziekrant,Poëziekrant,1573,2030-0638
...,...,...,...,...
4026,Broeker bijdragen,Broeker bijdragen,1,
4027,Doorbraak,Doorbraak,1,
4028,Collection d'art,Collection d'art,1,
4029,Pro Petri Sede,Pro Petri Sede,1,


In [46]:
jour.to_excel('../data/journal_titles.xlsx', header=True, index=False)